<a href="https://colab.research.google.com/github/Bmdhia/sales_predictions/blob/main/Project_1_Part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [131]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [132]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(transform_output='pandas')

In [133]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [134]:
df = pd.read_csv('/content/drive/MyDrive/sales_predictions_2023.csv')

In [135]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


* Before splitting your data, you can drop duplicates and fix inconsistencies in categorical dat :

In [136]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [137]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [138]:
df.duplicated().sum()

0

In [139]:
cat = df.select_dtypes(include=['object', 'category']).columns.tolist()

In [140]:
for col in cat:
        print(f"Column: {col}")
        print(df[col].value_counts())
        print("\n")

Column: Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64


Column: Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64


Column: Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64


Column: Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018  

In [141]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('LF',"Low Fat")

In [142]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('low fat',"Low Fat")

In [143]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('reg',"Regular")

* Identify the features (X) and target (y) :

In [144]:
X = df.drop(columns='Item_Outlet_Sales')
y = df['Item_Outlet_Sales']

* Perform a train test split :

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

* Create a preprocessing object to prepare the dataset for Machine Learning :

In [146]:
num_cols = X_train.select_dtypes('number').drop(columns='Outlet_Establishment_Year').columns
ord_cols = ['Outlet_Size']
nom_cols = X_train.select_dtypes('object').drop(columns=['Outlet_Size','Item_Identifier']).columns

In [147]:
#Numeric
scaler = StandardScaler()
impute_mean = SimpleImputer()
num_pipe = make_pipeline(impute_mean, scaler)

#Nominal
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

#Ordinal
impute_freq = SimpleImputer(strategy='most_frequent')
the_list = ['Small', 'High', 'Medium']
ord_encoder = OrdinalEncoder(categories=[the_list])
scaler_ord = StandardScaler()
ord_pipe = make_pipeline(impute_freq, ord_encoder, scaler_ord)

#Tuples

num_tuple = ('numeric', num_pipe, num_cols)
nom_tuple = ('Nominal', ohe_encoder, nom_cols)
ord_tuple = ('ordinal', ord_pipe, ord_cols)


In [148]:
col_transformer = ColumnTransformer([num_tuple, nom_tuple, ord_tuple],
                                    remainder='passthrough',
                                    verbose_feature_names_out=False)

In [150]:
col_transformer.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numeric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP'], dtype='object')),
                                ('Nominal',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')),
                                ('ordinal',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Small',
                                                                              'High',
                                                                              'Medium']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Outlet_Size'])],
                  verbose_feature_names_out=False)

In [153]:
roronoa = col_transformer.transform(X_train)

In [154]:
roronoa.describe()

,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Outlet_Identifier_OUT049,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Outlet_Size,Outlet_Establishment_Year
count,6.392000e+03,6.392000e+03,6.392000e+03,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,...,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6392.000000,6.392000e+03,6392.000000
mean,4.457566e-16,-6.169450e-17,4.668773e-17,0.645964,0.354036,0.074781,0.027378,0.013141,0.075250,0.079318,...,0.105757,0.274562,0.329474,0.395964,0.125000,0.651752,0.110138,0.113110,1.189425e-16,1997.857165
std,1.000078e+00,1.000078e+00,1.000078e+00,0.478258,0.478258,0.263058,0.163195,0.113889,0.263815,0.270255,...,0.307551,0.446328,0.470059,0.489095,0.330745,0.476452,0.313086,0.316752,1.000078e+00,8.392300
min,-1.980409e+00,-1.291052e+00,-1.767529e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.507624e+00,1985.000000
25%,-8.075039e-01,-7.624234e-01,-7.638272e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.507624e+00,1987.000000
50%,4.213344e-16,-2.318711e-01,3.400912e-02,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,7.547851e-01,1999.000000
75%,7.579511e-01,5.596016e-01,7.172910e-01,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,7.547851e-01,2004.000000
max,2.003199e+00,5.132050e+00,1.994559e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.547851e-01,2009.000000


In [156]:
roronoa.isnull().sum()

Item_Weight                        0
Item_Visibility                    0
Item_MRP                           0
Item_Fat_Content_Low Fat           0
Item_Fat_Content_Regular           0
Item_Type_Baking Goods             0
Item_Type_Breads                   0
Item_Type_Breakfast                0
Item_Type_Canned                   0
Item_Type_Dairy                    0
Item_Type_Frozen Foods             0
Item_Type_Fruits and Vegetables    0
Item_Type_Hard Drinks              0
Item_Type_Health and Hygiene       0
Item_Type_Household                0
Item_Type_Meat                     0
Item_Type_Others                   0
Item_Type_Seafood                  0
Item_Type_Snack Foods              0
Item_Type_Soft Drinks              0
Item_Type_Starchy Foods            0
Outlet_Identifier_OUT010           0
Outlet_Identifier_OUT013           0
Outlet_Identifier_OUT017           0
Outlet_Identifier_OUT018           0
Outlet_Identifier_OUT019           0
Outlet_Identifier_OUT027           0
O